# Exercícios Pandas parte 1

Agora, é hora de praticar tudo o que você aprendeu! Vamos simular um ambiente de trabalho aqui, portanto, além de fazer o que foi solicitado, é preciso que você busque formas de garantir que sua resposta está correta. Encontre formas de validar que o seu resultado está correto!

Utilizando o mesmo dataset do material, faça o seguinte:

1) Importe os dados e verifique as primeiras linhas do dataset. Verifique também as últimas linhas do dataset.

2) Verifique se existe algum valor missing no dataset.

3) Quanto de memória o dataset está consumindo?

4) Gere as estatísticas descritivas das colunas numéricas do dataset. Qual a mediana da coluna BMI? E a média? Qual desses valores é maior? O que isso significa? Caso não saiba o que isso significa, passe os 2 valores ao chatGPT e pergunte a ele o que significa a média ser maior ou menor que a mediana.

5) Qual a proporção de fumantes no nosso dataset? E de não fumantes?

6) Existe alguma diferença na cobrança para fumantes e não fumantes? Se sim, ao que você imagina que isso se deve?

7) Qual a média de BMI para fumantes? Ela é muito diferente da observada para fumantes? Se sim, por qual motivo você acha que isso ocorre?

8) Existe alguma diferença de cobrança para quem tem filhos? Identifique se ter filhos parece acarretar numa cobrança maior olhando a média de charges para quem tem 0 filhos e a média para quem tem 1 ou mais filhos.

9) Para o homem, o BMI de 20 a 24,9 é considerado normal. Acima disso, é considerado acima do peso, abaixo é considerado abaixo do peso. Crie uma nova coluna, chamada "bmi_male" que receberá o texto "abaixo do normal" para BMI abaixo de 20, "normal" para BMI de 20 a 24,9 e "acima do normal" para BMI acima de 24,9.

10) Se uma homem é fumante e ainda tem um BMI acima do considerado normal, é possível que isso seja um indicador negativo para a seguradora. Quantos homens estão nessa situação?

In [ ]:
# 1. Importe os dados e verifique as primeiras linhas do dataset. Verifique também as últimas linhas do dataset.

import kagglehub
import pandas as pd

path = kagglehub.dataset_download("mirichoi0218/insurance")
df = pd.read_csv("/kaggle/input/insurance/insurance.csv")
df.head()
df.tail()

In [ ]:
# 2. Verifique se existe algum valor missing no dataset.
df.isnull()

In [ ]:
# 3. Quanto de memória o dataset está consumindo?
df.info()

O dataset está utilizando 73.3 KB de memória

In [ ]:
# 4. Gere as estatísticas descritivas das colunas numéricas do dataset. 
# Qual a mediana da coluna BMI? E a média? Qual desses valores é maior? 
# O que isso significa? 
"""(Caso não saiba o que isso significa, passe os 2 valores ao chatGPT 
e pergunte a ele o que significa a média ser maior ou menor que a mediana.)"""

df.describe()

A mediana da columa BMI é 30.4 e a média é 30.663397 </br>
Quando comparamos as duas, estamos olhando se há simetria na curva. </br>
Se a média estiver maior que a mediana ou menor, deve haver outliers que estão puxando ela para cima ou para baixo. </br>
Quando os valores são próximos (como no caso), dizemos que eles são aproximadamente simétricos.

In [ ]:
proporcoes = df['smoker'].value_counts(normalize=True) * 100

print(f"O percentual de fumantes é {proporcoes['yes']:.2f}% e de não fumantes é {proporcoes['no']:.2f}%")


In [ ]:
# 6. Existe alguma diferença na cobrança para fumantes e não fumantes? 
# Se sim, ao que você imagina que isso se deve?
df.groupby('smoker')['charges'].mean()

Existe diferença e pode haver com o fato de quem fuma desenvolver doenças crônicas ao longo do tempo, gastando com remédios e tratamento.

In [ ]:
# 7. Qual a média de BMI para fumantes? Ela é muito diferente da observada para fumantes? 
# Se sim, por qual motivo você acha que isso ocorre?
df.groupby('smoker')['bmi'].mean()

Pela observação não uma diferença significativa do IMC entre fumantes e não fumantes.

In [ ]:
# 8. Existe alguma diferença de cobrança para quem tem filhos? 
"""(Identifique se ter filhos parece acarretar numa cobrança maior olhando 
a média de charges para quem tem 0 filhos e a média para quem tem 1 ou mais filhos.)"""

df['tem_filhos'] = df['children'] > 0
media_filho = df.groupby('tem_filhos')['charges'].mean()

sem_filho = media_filho[False]
com_filho = media_filho[True]

print(f"A média para quem tem filho é R${com_filho:,.2f}")
print(f"A média para quem não tem filho é R${sem_filho:,.2f}")

In [ ]:
"""
9. Para o homem, o BMI de 20 a 24,9 é considerado normal. 
Acima disso, é considerado acima do peso, abaixo é considerado abaixo do peso. 
Crie uma nova coluna, chamada "bmi_male" que receberá o texto "abaixo do normal" 
para BMI abaixo de 20, "normal" para BMI de 20 a 24,9 
e "acima do normal" para BMI acima de 24,9.
"""

df['bmi_male'] = None
df.loc[df['sex'] == 'male', 'bmi_male'] = df.loc[df['sex'] == 'male', 'bmi'].apply(
    lambda x: 'abaixo do normal' if x < 20
    else ('acima do normal' if x > 24.9 else 'normal' )
)
df['bmi_male'].head(20)

In [ ]:
# 10. Se uma homem é fumante e ainda tem um BMI acima do considerado normal, 
# é possível que isso seja um indicador negativo para a seguradora. 
# Quantos homens estão nessa situação?

bmi_male_acima = df.loc[(df['sex'] == 'male') & (df['smoker'] == 'yes') & (df['bmi'] > 24.9)]
bmi_male_acima.shape[0]